In [ ]:
!pip install arxiv
!pip install wikipedia

In [3]:
## Arxiv -- Research Paper Tool
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [5]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wikipedia.name

'wikipedia'

In [ ]:
## Used the inbuit tool for Arxiv
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [7]:
tools=[wikipedia,arxiv]

## Custom Tools

In [9]:
#Custom tools/RAG tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [12]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=5000,chunk_overlap=500)
documents=text_splitter.split_documents(docs)

In [14]:
vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings())
vectordb

In [15]:
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019D470B18A0>, search_kwargs={})

In [16]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"Langsmit-search","Search any info about langsmit")
retriever_tool.name

'Langsmit-search'

In [17]:
tools=[wikipedia,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\DS_ML_AI_DL_GENAI_practice\\LangChain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Langsmit-search', description='Search any info about langsmit', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000019D5953F640>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019D470B1

In [ ]:
### Running all tools with agents and LLM model
import os
from dotenv import load_dotenv
load_dotenv()

LANGCHAIN_API_KEY=os.environ.get("LANGCAHIN_API_KEY")
HF_TOKEN=os.environ.get("HF_TOKEN")
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")

In [19]:
from langchain_groq import ChatGroq
llm=ChatGroq(api_key=GROQ_API_KEY,model="Llama3-8b-8192")

In [22]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

e:\DS_ML_AI_DL_GENAI_practice\LangChain\venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [23]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [25]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [26]:
agent_executor.invoke({"input":"Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `Langsmit-search` with `{'query': 'Langsmit'}`


Get started with LangSmith | 🦜️🛠️ LangSmith








Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in 

{'input': 'Tell me about langsmith',
 'output': "Here's the response:\n\nLangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. LangSmith has several features, including observability, evals, and prompt engineering."}

In [27]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witBased on the result of the tool call, I can provide a direct response without using any additional tools.

Machine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time.

> Finished chain.


{'input': 'What is machine learning',
 'output': 'Based on the result of the tool call, I can provide a direct response without using any additional tools.\n\nMachine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time.'}

In [28]:
agent_executor.invoke({"input":"arCiv:1706.03762"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'arCiv:1706.03762'}`


No good Arxiv Result was found
Invoking: `wikipedia` with `{'query': 'No good Arxiv Result was found'}`


Page: Hallucination (artificial intelligence)
Summary: In the field of artificial intelligence (AI), a hallucination or artificial hallucination (also called bullshitting, confabulation or delusion) is a response generated by AI that contains false o
Invoking: `wikipedia` with `{'query': 'artificial intelligence'}`


Page: Artificial intelligence
Summary: Artificial intelligence (AI) refers to the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decisioThe tool calls have yielded sufficient information.

> Finished chain.


{'input': 'arCiv:1706.03762',
 'output': 'The tool calls have yielded sufficient information.'}